## I. Import Libraries & Data.

In [1]:
import os
os.chdir('/Users/Rho.l/Desktop/NLP')
os.getcwd()

'/Users/Rho.l/Desktop/NLP'

In [60]:
import pandas as pd
import nltk
import re
import string
import sklearn

pd.set_option('display.max_colwidth',100)

#ps = nltk.PorterStemmer() I think i can cut this out.
wnL = nltk.WordNetLemmatizer()

#nltk.download('wordnet')
#nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')


data = pd.read_csv('conference_talks_2021.csv',sep=',', encoding='unicode_escape')
#data = pd.read_csv('conference_talks_2021.csv',sep=',')


data.head()

,title,speaker,calling,conference,url,conference_talk
0,"Pure Truth, Pure Doctrine, and Pure Revelation",By President RussellÂ M. Nelson,President of The Church of Jesus Christ of Latter-day Saints,21-Oct,https://www.churchofjesuschrist.org/study/general-conference/2021/10/11nelson,"my dear brothers and sisters, welcome to general conference! what a joy it is to be with you! yo..."
1,The Greatest Possession,By Elder JeffreyÂ R. Holland,Of the Quorum of the Twelve Apostles,21-Oct,https://www.churchofjesuschrist.org/study/general-conference/2021/10/12holland,"the scriptures speak of a rich young ruler who ran to jesus, knelt at his feet, and, with genuin..."
2,Come unto Christ and Donât Come Alone,By BonnieÂ H. Cordon,Young Women General President,21-Oct,https://www.churchofjesuschrist.org/study/general-conference/2021/10/13cordon,i recently received a letter from an inquisitive young woman. she wrote: âi am stuck.Â â¦ iâ...
3,The Saviorâs Abiding Compassion,By Elder Ulisses Soares,Of the Quorum of the Twelve Apostles,21-Oct,https://www.churchofjesuschrist.org/study/general-conference/2021/10/14soares,one of the most striking principles taught by the savior during his earthly ministry was treatin...
4,The Love of God,By Elder D.Â Todd Christofferson,Of the Quorum of the Twelve Apostles,21-Oct,https://www.churchofjesuschrist.org/study/general-conference/2021/10/15christofferson,"our heavenly father loves us profoundly and perfectly.1 in his love, he created a plan, a plan o..."


In [3]:
## II. CLEAN & TOKENIZE MY DATA
    ##Clean_text generates two variables: text, token_list. 'Text' is vestigial. I held onto the code just in case.
    ## 'text' gives me the list of tokens in a string.
    ## 'token_list' gives me the final list of tokens in an array. I prefer to work with the array.

def clean_text(text):
    
    text = "".join(word.lower() for word in text if word not in string.punctuation)
    tokens = re.split('\W+', text)
    
    #text = " ".join([wnL.lemmatize(word) for word in tokens if word not in stopwords])
    token_list = [wnL.lemmatize(word) for word in tokens if word not in stopwords]
    
    return token_list


desc_clean = data['talk'].apply(lambda x:clean_text(x))
print (desc_clean.head(5))

    ## I don't want to add this to my datatable because this is an intermediate v. 
    ## We are going to turn this list into a list of ngrams.


0    [dear, brother, sister, welcome, general, conference, joy, mind, almost, constantly, past, six, ...
1    [scripture, speak, rich, young, ruler, ran, jesus, knelt, foot, genuine, sincerity, asked, maste...
2    [recently, received, letter, inquisitive, young, woman, wrote, â, stuckâ, â, iâ, sure, feel, iâ,...
3    [one, striking, principle, taught, savior, earthly, ministry, treating, others, compassion, let,...
4    [heavenly, father, love, u, profoundly, perfectly1, love, created, plan, plan, redemption, happi...
Name: talk, dtype: object


In [4]:
## III. CREATE NGRAM FREQUENCY TABLE
    ##N-GRAM CREATOR
    ##takes two inputs: a list of tokens, and the 'n' we want to generate nGrams for 

def find_ngrams(token_list, n):
    
    ngram_array =[list(zip(*[token_list[i:] for i in range(n)]))]
    
    ngram_list = []
    
    for ngram in ngram_array:
        for word in ngram:
            ngram_list.append(" ".join(word))
    
        #text = " ".join([wnL.lemmatize(word) for word in tokens if word not in stopwords])     
        
    return ngram_list


##CREATE 'desc_ngrams'
## This v creates a list of lists. It gives me a list of trigrams and bigrams for each description.
## I want this list broken out by description (desc). 

desc_ngrams = []

for desc in desc_clean:
    desc_ngrams.append(find_ngrams(desc,1) + find_ngrams(desc,2)+ find_ngrams(desc,3))
    
desc_ngrams    

[['dear',
  'brother',
  'sister',
  'welcome',
  'general',
  'conference',
  'joy',
  'mind',
  'almost',
  'constantly',
  'past',
  'six',
  'month',
  'prayed',
  'recent',
  'week',
  'prayed',
  'intently',
  'conference',
  'would',
  'time',
  'revelation',
  'reflection',
  'seek',
  'blessing',
  'delighted',
  'speaking',
  'conference',
  'center',
  'seat',
  'remain',
  'empty',
  'presence',
  'member',
  'tabernacle',
  'choir',
  'wonderful',
  'step',
  'forward',
  'welcome',
  'largely',
  'virtual',
  'conference',
  'wherever',
  'still',
  'dealing',
  'ravage',
  'covid19',
  'variant',
  'thank',
  'following',
  'counsel',
  'advice',
  'medical',
  'expert',
  'government',
  'official',
  'community',
  'convene',
  'general',
  'conference',
  'directed',
  'lord1',
  'format',
  'varied',
  'year',
  'much',
  'younger',
  'conference',
  'lasted',
  'three',
  'four',
  'day',
  'later',
  'conference',
  'reduced',
  'two',
  'day',
  'every',
  'messag

In [6]:
## BUILD A FREQUENCY TABLE
## This func takes a list of words (or ngrams). It creates a frequency list

def build_frequency_table(corpus):
    frequency_list = [corpus.count(word) for word in corpus]
    frequency_data = {key:value for key, value in zip(corpus, frequency_list)}
    sorted_data = sorted(frequency_data.items(), key=lambda kv: kv[1], reverse=True)
    frequency_table = {key:value for key,value in sorted_data}
    return frequency_table

## all_ngrams. I need to collect all the ngrams across every description for a combined frequency table.
    

all_ngrams = []

for ngram in desc_ngrams:
    for word in ngram:
        all_ngrams.append(word)

        
ngram_frequency_table = build_frequency_table(all_ngrams)


for key,value in ngram_frequency_table.items():
    print (key,value)
    
    ##This is my first draft of the final frequency table. 
    ## Next step is to generate a library of similar words that are specific to this corpus
    ## For example, jesus, jesus christ, savior, chirst, lord, are all similar in meaning.
    ## It would be helpful to see this combined with the original file.

â 819
u 380
christ 358
jesus 331
god 266
jesus christ 248
love 212
lord 204
life 192
church 191
savior 151
one 143
faith 140
time 134
thing 116
father 114
gospel 112
know 109
even 105
president 102
temple 100
may 97
would 96
covenant 92
come 91
many 91
day 90
work 89
unto 88
heart 87
family 87
godâ 87
way 85
name 84
people 83
upon 83
world 81
great 81
need 79
help 77
first 76
brother 74
power 73
shall 73
said 70
child 68
prophet 67
member 66
year 65
young 65
become 65
receive 64
word 64
make 61
peace 61
heavenly 61
spiritual 60
feel 60
taught 60
nelson 60
also 60
could 59
holy 59
others 59
scripture 58
joy 56
see 56
home 56
sister 55
blessing 55
like 55
answer 55
question 54
saint 54
nephi 52
never 51
give 51
keep 49
asked 48
want 48
heavenly father 48
experience 48
son 47
go 46
ask 46
ye 46
seek 45
every 45
truth 45
good 45
mormon 45
saviorâ 45
let 45
missionary 45
man 44
take 44
change 43
commandment 43
sin 43
small 43
made 43
true 43
name jesus 42
name jesus christ 42
hand 41
person

reflect 5
simon 5
luke 5
lack 5
shown 5
evidence 5
countless 5
expression 5
intense 5
instrument 5
problem 5
increased 5
whose 5
wise 5
witness jesus 5
remission sin 5
atoning sacrifice 5
difficult circumstance 5
dear friend 5
redemption 5
achieve 5
universal 5
run 5
r 5
final 5
anyone 5
anxiety 5
deeply 5
26 5
bishop 5
oh 5
mark 5
constant 5
savior â 5
guide u 5
said unto 5
â anything 5
prophet god 5
â thou 5
lord â 5
see 1â 5
happiness peace 5
peace joy 5
professional 5
sign 5
standard 5
intercept 5
low 5
endowment 5
advantage 5
access 5
remarkable 5
overwhelmed 5
realize 5
quoted 5
fire 5
pioneer 5
returning 5
successful 5
thatâ 5
sight 5
son daughter 5
mention 5
touched 5
gradually 5
america 5
accepted 5
forward faith 5
god heavenly 5
invite u 5
god heavenly father 5
talk 5
honest 5
include 5
number 5
waiting 5
usually 5
reminds 5
membership 5
perform 5
procedure 5
happen 5
otherwise 5
addition 5
tradition 5
caring 5
obey 5
original 5
remind 5
million 5
individually 5
church attend

seventh 3
elisha 3
river 3
clean 3
april 3
commit 3
peter â 3
thing lord 3
youâ â 3
asking question 3
responded â 3
commandment â 3
thy soul 3
love thy 3
ye love 3
love keep 3
keep commandmentsâ 3
little better 3
attend temple 3
thing matter 3
good cheer 3
14 lord 3
follow counsel 3
remind u 3
river jordan 3
counsel president 3
christ many 3
â thing soulâ 3
â love thy 3
lord said â 3
jesus christ many 3
jacob 3
consequently 3
agree 3
mankind 3
lover 3
regard 3
inspiring 3
changing 3
growing 3
rarely 3
shadow 3
conclude 3
six year 3
mormon prophet 3
god hath 3
behold say 3
meâ 9 3
holy spirit 3
â earth 3
lord god 3
gospel repentance 3
revelation god 3
behold say unto 3
passing 3
lazarus 3
temporary 3
tragic 3
imagined 3
highly 3
daysâ 3
included 3
jerusalem 3
possession 3
broke 3
ship 3
treat 3
matthew 3
village 3
hinckley 3
worry 3
one many 3
many trial 3
dark day 3
â feel 3
joy regardless 3
â livesâ 3
nephi experienced 3
disciple came 3
jesus went 3
jesus said 3
difficult trial 3
ever

upon blessing 2
eternal principle 2
u closer 2
â felt 2
eternal companion 2
crown glory 2
million people 2
good work 2
man cometh 2
u strength 2
disciple christ 2
god name 2
church help u 2
u become better 2
attending church meeting 2
saviorâ restored church 2
authoritative priesthood ordinance 2
draw u closer 2
truth â spirit 2
give u strength 2
god name jesus 2
vote 2
dieterâ 2
uchtdorf 2
l 2
daleâ 2
renlund 2
released 2
jâ 2
jamesâ 2
davy 2
authority area 2
president church 2
second counselor 2
president quorum 2
apostle mâ 2
following member 2
dieterâ f 2
f uchtdorf 2
daleâ g 2
g renlund 2
express gratitude 2
gratitude brother 2
service may 2
general authority area 2
manifest opposed may 2
opposed may manifest 2
president quorum twelve 2
twelve apostle mâ 2
apostle mâ russell 2
dieterâ f uchtdorf 2
daleâ g renlund 2
please manifest opposed 2
1823 2
messenger 2
christ3 2
preaching 2
introduction 2
initiated 2
version 2
plant 2
utterly 2
wasted 2
doctrinal 2
aprilâ 2
duty 2
lightâ 2


draw near 2
people lost 2
walk circle 2
max planck 2
walk straight 2
visible landmark 2
test subject 2
high confidence 2
walking straight 2
straight line 2
small seemingly 2
seemingly insignificant 2
straight ahead 2
5 whatever 2
whatever cause 2
difference life 2
applies u 2
place never 2
thought would 2
yet work 2
course take 2
strong spiritual 2
experience past 2
daily prayer 2
lord instructed 2
spiritual landmark 2
u saviorâ 2
heavenly light 2
doubt seek 2
need ongoing 2
time one 2
go life 2
want change 2
change shape 2
u child 2
become new 2
new creature 2
15 â 2
sister dear 2
jesus christ hope 2
walking straight line 2
small seemingly insignificant 2
5 whatever cause 2
spiritual experience past 2
â god shall 2
gift holy ghostâ 2
become new creature 2
sister dear friend 2
storyâ 2
lawyer 2
resistant 2
plot 2
timeless 2
giant 2
tending 2
lion 2
paw 2
pencil 2
persia 2
mordecai 2
estherâ 2
several question 2
heavenly home 2
story know 2
heaven earth 2
trial lawyer 2
witness question

...

momentous moment 1
moment life 1
life ready 1
ready right 1
right assist 1
assist whatever 1
whatever 1â 1
percent quest 1
quest coming 1
coming conference 1
conference every 1
every effort 1
effort change 1
change makeâ 1
makeâ matter 1
matter tiny 1
tiny seems 1
seems usâ 1
usâ might 1
might make 1
make biggest 1
biggest difference 1
life end 1
end elder 1
â assertion 1
assertion righteous 1
righteous desire 1
desire act 1
act service 1
service act 1
act worship 1
worship however 1
however small 1
small incremental 1
incremental add 1
add spiritual 1
spiritual momentumâ 1
momentumâ 14 1
14 truly 1
truly small 1
simple yes 1
even 1â 1
percent thing 1
brought pass15 1
pass15 ultimate 1
ultimate victory 1
victory 100â 1
100â percent 1
certain â 1
â doâ 1
doâ 16 1
16 might 1
might merit 1
merit mercy 1
mercy lord 1
testify name 1
century national bicycle 1
national bicycle racing 1
bicycle racing team 1
racing team great 1
team great britain 1
great britain laughingstock 1
britain laughi

young powerful 1
letting circumstance 1
circumstance determine 1
determine happiness 1
happiness always 1
always happy 1
happy last 1
last may 1
may ruby 1
ruby faced 1
faced additional 1
additional hurricane 1
hurricane life 1
life faith 1
also born 1
born le 1
le fully 1
fully developed 1
developed hand 1
needed reconstructive 1
reconstructive surgery 1
surgery prior 1
prior rather 1
rather complex 1
complex operation 1
operation visited 1
visited gave 1
gave drawing 1
drawing beautifully 1
beautifully depicts 1
depicts childâ 1
childâ hand 1
hand warmly 1
warmly holding 1
holding hand 1
hand savior 1
savior asked 1
asked nervous 1
nervous replied 1
â happyâ 1
happyâ asked 1
â ruby 1
ruby soâ 1
soâ ruby 1
ruby confidently 1
confidently asserted 1
asserted â 1
jesus hold 1
hold handâ 1
handâ rubyâ 1
rubyâ recovery 1
recovery miraculous 1
miraculous continues 1
continues happy 1
happy purity 1
purity childâ 1
childâ faith 1
faith contrast 1
contrast foolishness 1
foolishness doubt 1
do

soap repentance 1
repentance cleaning 1
cleaning agent 1
agent allows 1
opportunity get 1
get rid 1
rid impurity 1
impurity old 1
old debris 1
debris worthy 1
worthy god 1
thing inherit 1
godâ alma 1
alma 1137 1
1137 repentance 1
repentance draw 1
upon cleansing 1
cleansing sanctifying 1
key part 1
part process 1
process conversion 1
conversion happened 1
happened antinephilehies 1
antinephilehies book 1
mormon lamanites 1
lamanites completely 1
completely converted 1
converted â 1
never fall 1
fall awayâ 1
awayâ see 1
alma 236â 1
236â 8 1
8 buried 1
buried weapon 1
weapon war 1
war never 1
never took 1
took would 1
would rather 1
rather die 1
die break 1
break covenant 1
covenant proved 1
proved know 1
know sacrifice 1
sacrifice brought 1
brought miracle 1
miracle thousand 1
thousand fought 1
fought threw 1
threw weapon 1
weapon converted 1
converted year 1
later son 1
son know 1
know mighty 1
mighty stripling 1
stripling warrior 1
warrior protected 1
protected battle 1
battle incredi

lose hope 1
rescue always 1
always come 1
come faithful 1
faithful whether 1
whether mortality 1
mortality eternity 1
eternity sometimes 1
sometimes sinking 1
sinking come 1
come mistake 1
mistake transgression 1
transgression find 1
sinking reason 1
reason make 1
make joyful 1
joyful choice 1
choice repent13 1
repent13 believe 1
thing give 1
give savior 1
joy saving 1
saving turn 1
turn return 1
return him14 1
him14 scripture 1
scripture full 1
full story 1
story people 1
people fallen 1
fallen flawed 1
flawed repented 1
repented became 1
became firm 1
firm faith 1
christ think 1
think story 1
story scripture 1
scripture remind 1
power redeem 1
redeem u 1
u infinite 1
infinite savior 1
joy repent 1
repent receive 1
receive great 1
joy well 1
well invite 1
invite intentional 1
intentional â 1
roadâ increase 1
increase focus 1
matter may 1
keep christ 1
focus midst 1
midst distraction 1
distraction thing 1
hoodâ whirlwind 1
whirlwind surround 1
surround u 1
jesus savior 1
redeemer rescu

language new 1
new domain 1
domain name 1
name acquired 1
acquired across 1
world appreciate 1
appreciate many 1
many good 1
good gracious 1
gracious people 1
people honored 1
honored desire 1
called correct 1
name read 1
read article 1
article recently 1
recently quoted 1
quoted catholic 1
catholic cardinal 1
cardinal referring 1
referring â 1
â latterday 1
saintsâ 18 1
18 visited 1
visited leader 1
leader christian 1
christian church 1
church month 1
month ago 1
ago eastern 1
state referred 1
referred church 1
church first 1
first reference 1
reference entire 1
entire name 1
name followed 1
followed â 1
christâ realized 1
realized adding 1
adding six 1
six word 1
word name 1
name would 1
would ideal 1
ideal medium 1
medium president 1
nelson foretold 1
foretold â 1
â responsible 1
responsible medium 1
medium sympathetic 1
sympathetic responding 1
responding requestâ 1
requestâ 19 1
19 thank 1
thank extending 1
extending u 1
u consideration 1
consideration given 1
given cultural 1
cul

In [12]:
ngram_list = pd.DataFrame.from_dict(ngram_frequency_table, orient="index", columns=['frequency'])
#ngram_list.to_csv("ngram_list_raw.csv")

In [ ]:
def calculate_count_of_talks(n_gram, talk):
    i = 0
    for talk in data.talk:
        if talk.count(ngram_list.iloc[[2]].index[0]) > 0:
            i = i+1
    i

In [35]:
count_list = [talk.count(ngram_list.iloc[[2]].index[0]) for talk in data.talk]
#count_list

In [16]:
ngram_list.iloc[[2]].index[0]

'christ'

In [56]:
def add_n_gram_column(n_gram_str):
    data[str(n_gram_str)] = data.apply(lambda row: row.conference_talk.count(str(n_gram_str)), axis=1)

In [59]:
ngram_list.index[:2000]

Index(['â', 'u', 'christ', 'jesus', 'god', 'jesus christ', 'love', 'lord',
       'life', 'church',
       ...
       'called â', 'dwell heart', 'could happier', 'happier people',
       'created hand', 'â blessed', 'pas away', 'great truth', 'second great',
       'â could'],
      dtype='object', length=2000)

In [61]:
for index in ngram_list.index[:2000]:
    add_n_gram_column(str(index))
data.head()

,title,speaker,calling,conference,url,conference_talk,â,u,christ,jesus,...,called â,dwell heart,could happier,happier people,created hand,â blessed,pas away,great truth,second great,â could
0,"Pure Truth, Pure Doctrine, and Pure Revelation",0,0,10,https://www.churchofjesuschrist.org/study/general-conference/2021/10/11nelson,"my dear brothers and sisters, welcome to general conference! what a joy it is to be with you! yo...",5,92,7,4,...,0,0,0,0,0,0,0,0,0,0
1,The Greatest Possession,0,0,1,https://www.churchofjesuschrist.org/study/general-conference/2021/10/12holland,"the scriptures speak of a rich young ruler who ran to jesus, knelt at his feet, and, with genuin...",74,162,7,5,...,0,0,0,2,0,0,0,1,1,0
2,Come unto Christ and Donât Come Alone,0,0,1,https://www.churchofjesuschrist.org/study/general-conference/2021/10/13cordon,i recently received a letter from an inquisitive young woman. she wrote: âi am stuck.Â â¦ iâ...,62,170,18,7,...,0,0,0,0,0,0,0,0,0,0
3,The Saviorâs Abiding Compassion,0,0,0,https://www.churchofjesuschrist.org/study/general-conference/2021/10/14soares,one of the most striking principles taught by the savior during his earthly ministry was treatin...,45,210,4,21,...,0,0,0,0,0,0,0,0,0,0
4,The Love of God,0,0,0,https://www.churchofjesuschrist.org/study/general-conference/2021/10/15christofferson,"our heavenly father loves us profoundly and perfectly.1 in his love, he created a plan, a plan o...",93,205,13,9,...,0,0,0,0,0,0,0,0,0,0


In [62]:
data.to_csv("general_conference_analysis.csv")